# Model Experimentation Tracking (MLFow) - Hyperparamter Optimization

Record and query experiments: Code, data, config, results, parameters, metrics

![Data](images/MLflow_Model_experimentation.png)

## Import Packages

In [1]:
# Data analysis library
import numpy as np
import pandas as pd
import joblib

# Machine Learning library
import sklearn
from sklearn.metrics import roc_curve, auc, accuracy_score, plot_confusion_matrix, plot_roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import plot_importance, plot_metric

# Model experimentation library
import mlflow
import mlflow.lightgbm
from mlflow.tracking import MlflowClient

# Hyperparameter tunning library
import optuna

# Plotting library
import matplotlib.pyplot as plt
# Prevent figures from displaying by turning interactive mode off using the function
plt.ioff()
import warnings
warnings.filterwarnings("ignore")

In [2]:
print(f'Numpy version is {np.__version__}')
print(f'Pandas version is {pd.__version__}')
print(f'sklearn version is {sklearn.__version__}')
print(f'mlflow version is {mlflow.__version__}')
print(f'joblib version is {joblib.__version__}')
print(f'optuna version is {optuna.__version__}')

Numpy version is 1.19.4
Pandas version is 1.1.5
sklearn version is 0.23.2
mlflow version is 1.12.1
joblib version is 1.0.0
optuna version is 2.3.0


## Download data 

### Campus Recruitment Dataset
#### Academic and Employability Factors influencing placement

https://www.kaggle.com/benroshan/factors-affecting-campus-placement

## Load data

In [3]:
## Files
data_file = '../data/placement_data/Placement_Data_Full_Class.csv'

# Load train loan dataset 
try:
    data = pd.read_csv(data_file)
    print("The dataset has {} samples with {} features.".format(*data.shape))
except:
    print("The dataset could not be loaded. Is the dataset missing?")

The dataset has 215 samples with 15 features.


## Introduction To The Data

In [4]:
data.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


In [5]:
data['status'].value_counts()

Placed        148
Not Placed     67
Name: status, dtype: int64

## Start MLflow UI

Start **mlflow ui** comman from the command prompt

In [6]:
!mlflow ui

^C


## Initialize MLflow

**Experiments** : You can organize runs into experiments, which group together runs for a specific task. 

**Tracking URI**: MLflow runs can be recorded to local files, to a database, or remotely to a tracking server. By default, the MLflow Python API logs runs locally to files in an mlruns directory wherever you ran your program

#### MLflow Tracking Servers 
MLflow tracking server has two components for storage: a **backend store** and an **artifact store**

The **backend store** is where MLflow Tracking Server stores experiment and run metadata as well as params, metrics, and tags for runs. MLflow supports two types of backend stores: **file store and database-backed store**.

The **artifact store** is a location suitable for large data (such as an S3 bucket or shared NFS file system) and is where clients log their artifact output (for example, models).

    Amazon S3 and S3-compatible storage
    Azure Blob Storage
    Google Cloud Storage
    FTP server
    SFTP Server
    NFS
    HDFS

In [6]:
experiment_name = "campus_recruitment_experiments_v2"
artifact_repository = './mlflow-run'

# Provide uri and connect to your tracking server
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

# Initialize client
client = MlflowClient()

# If experiment doesn't exist then it will create new
# else it will take the experiment id and will use to to run the experiments
try:
    # Create experiment 
    experiment_id = client.create_experiment(experiment_name, artifact_location=artifact_repository)
except:
    # Get the experiment id if it already exists
    experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

## Prepare data for model training

In [7]:
exclude_feature = ['sl_no', 'salary', 'status']
# Define Target columns
target = data['status'].map({"Placed": 0 , "Not Placed": 1})

# Define numeric and categorical features
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
numeric_features = [col for col in numeric_columns if col not in exclude_feature]
categorical_features = [col for col in categorical_columns if col not in exclude_feature]

# Define final feature list for training and validation
features = numeric_features + categorical_features
# Final data for training and validation
data = data[features]
data = data.fillna(0)

# Split data in train and vlaidation
X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.15, random_state=10)

# Perform label encoding for categorical variable
for feature in categorical_features:
    le = LabelEncoder()
    le.fit(X_train.loc[:, feature])
    X_train.loc[:, feature] = le.transform(X_train.loc[:, feature])
    X_valid.loc[:, feature] = le.transform(X_valid.loc[:, feature])

## Lightgbm Hyperparameter tunning + MLFlow for model tracking

### Define model training function to train and track model results

In [8]:
def model_training_tracking(params):
    # Launching Multiple Runs in One Program.This is easy to do because the ActiveRun object returned by mlflow.start_run() is a 
    # Python context manager. You can “scope” each run to just one block of code as follows:
    with mlflow.start_run(experiment_id=experiment_id, run_name='Lightgbm_model') as run:
        # Get run id 
        run_id = run.info.run_uuid
        
        # Set the notes for the run
        MlflowClient().set_tag(run_id,
                               "mlflow.note.content",
                               "This is experiment for hyperparameter optimzation for lightgbm models for the Campus Recruitment Dataset")
        
        # Define customer tag
        tags = {"Application": "Payment Monitoring Platform",
                "release.candidate": "PMP",
                "release.version": "2.2.0"}

        # Set Tag
        mlflow.set_tags(tags)
                        
        # Log python environment details
        mlflow.log_artifact('requirements.txt')
        
        # logging params
        mlflow.log_params(params)

        # Perform model training
        lgb_clf = LGBMClassifier(**params)
        lgb_clf.fit(X_train, y_train, 
                    eval_set = [(X_train, y_train), (X_valid, y_valid)], 
                    early_stopping_rounds=50,
                    verbose=20)

        # Log model artifacts
        mlflow.sklearn.log_model(lgb_clf, "model")

        # Perform model evaluation 
        lgb_valid_prediction = lgb_clf.predict_proba(X_valid)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_valid, lgb_valid_prediction)
        roc_auc = auc(fpr, tpr) # compute area under the curve
        print("=====================================")
        print("Validation AUC:{}".format(roc_auc))
        print("=====================================")   

        # log metrics
        mlflow.log_metrics({"Validation_AUC": roc_auc})

        # Plot and save feature importance details
        ax = plot_importance(lgb_clf, height=0.4)
        filename = './images/lgb_validation_feature_importance.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)

        ax = plot_metric(lgb_clf.evals_result_)
        filename = './images/lgb_validation_metrics_comparision.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)

        # Plot and save metrics details    
        plot_confusion_matrix(lgb_clf, X_valid, y_valid, 
                              display_labels=['Placed', 'Not Placed'],
                              cmap='magma')
        plt.title('Confusion Matrix')
        filename = './images/lgb_validation_confusion_matrix.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)

        # Plot and save AUC details  
        plot_roc_curve(lgb_clf, X_valid, y_valid, name='Validation')
        plt.title('ROC AUC Curve')
        filename = './images/lgb_validation_roc_curve.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)
        
        return roc_auc

### Define an objective function to be maximized

In [9]:
def objective(trial):

    param = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": trial.suggest_float("learning_rate", 1e-2, 1e-1, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "random_state": 42,
    }
    
    auc = model_training_tracking(param)
    return auc

### Create a study object and optimize the objective function

In [10]:
# Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)
trial = study.best_trial
print('AUC: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-12-20 16:47:54,563] A new study created in memory with name: no-name-8668af4e-34dd-4a71-aad4-990b78ad4b0b


Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.974331	valid_1's auc: 0.87218
[40]	training's auc: 0.979743	valid_1's auc: 0.902256
[60]	training's auc: 0.986983	valid_1's auc: 0.894737
[80]	training's auc: 0.990493	valid_1's auc: 0.902256
Early stopping, best iteration is:
[38]	training's auc: 0.981205	valid_1's auc: 0.909774
Validation AUC:0.9097744360902256


[I 2020-12-20 16:47:56,361] Trial 0 finished with value: 0.9097744360902256 and parameters: {'learning_rate': 0.02701990132762442, 'colsample_bytree': 0.6804278072196543, 'subsample': 0.8445415704095669}. Best is trial 0 with value: 0.9097744360902256.


Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.978646	valid_1's auc: 0.868421
[40]	training's auc: 0.991663	valid_1's auc: 0.883459
Early stopping, best iteration is:
[2]	training's auc: 0.941568	valid_1's auc: 0.911654
Validation AUC:0.9116541353383458


[I 2020-12-20 16:47:57,384] Trial 1 finished with value: 0.9116541353383458 and parameters: {'learning_rate': 0.05806050035338419, 'colsample_bytree': 0.45345881652972353, 'subsample': 0.68724782238422}. Best is trial 1 with value: 0.9116541353383458.


...

Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.964385	valid_1's auc: 0.860902
[40]	training's auc: 0.973307	valid_1's auc: 0.868421
[60]	training's auc: 0.976964	valid_1's auc: 0.887218
[80]	training's auc: 0.979011	valid_1's auc: 0.894737
[100]	training's auc: 0.983619	valid_1's auc: 0.906015
Did not meet early stopping. Best iteration is:
[98]	training's auc: 0.983765	valid_1's auc: 0.902256
Validation AUC:0.9022556390977443


[I 2020-12-20 16:48:40,882] Trial 38 finished with value: 0.9022556390977443 and parameters: {'learning_rate': 0.01285573635448991, 'colsample_bytree': 0.6981101641465832, 'subsample': 0.598703668098904}. Best is trial 5 with value: 0.9210526315789475.


Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.977695	valid_1's auc: 0.894737
[40]	training's auc: 0.988738	valid_1's auc: 0.890977
[60]	training's auc: 0.994149	valid_1's auc: 0.902256
[80]	training's auc: 0.997367	valid_1's auc: 0.921053
[100]	training's auc: 0.999269	valid_1's auc: 0.913534
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.999269	valid_1's auc: 0.913534
Validation AUC:0.9135338345864661


[I 2020-12-20 16:48:42,135] Trial 39 finished with value: 0.9135338345864661 and parameters: {'learning_rate': 0.044358476903108804, 'colsample_bytree': 0.7617131228746111, 'subsample': 0.7467523870939182}. Best is trial 5 with value: 0.9210526315789475.


AUC: 0.9210526315789475
Best hyperparameters: {'learning_rate': 0.04980680386446043, 'colsample_bytree': 0.6734515470523221, 'subsample': 0.4965623082097034}


## Load best lightgbm model

Check Mlflow UI and pick the best model for model deployment

In [11]:
# Load best model
lgb_best_model = mlflow.sklearn.load_model("./mlflow-run/6ffb67cfcd87414a961bcdd0b69c04b0/artifacts/model")

# Make prediction aganist Validation data
lgb_best_val_prediction = lgb_best_model.predict(X_valid)
lgb_best_val_prediction

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int64)

## Reference

### Model experimentation
https://www.mlflow.org/docs/latest/tracking.html#

### Hyperparameter Optimization
https://github.com/optuna/optuna